In [1]:
%pylab
%matplotlib inline
%load_ext watermark
%watermark -v -m -p scipy,numpy,sklearn,tensorflow

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
CPython 3.5.2
IPython 5.1.0

scipy 0.18.1
numpy 1.11.2
sklearn 0.18
tensorflow 0.10.0

compiler   : GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)
system     : Darwin
release    : 16.1.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [2]:
from sklearn.datasets import load_digits
digits = load_digits()
digits_data = digits.data / 16

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits_data, digits.target, test_size=0.1)

In [4]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(n_values=10)
y_train_enc = ohe.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_enc = ohe.fit_transform(y_test.reshape(-1, 1)).toarray()
y_train_enc.shape, y_test_enc.shape

((1617, 10), (180, 10))

In [5]:
import tensorflow as tf

In [14]:
X = tf.placeholder(tf.float32, [None, 64])
X_input = tf.reshape(X, [-1, 8, 8, 1])
Y = tf.placeholder(tf.float32, [None, 10])

W_1 = tf.Variable(tf.random_normal([3, 3, 1, 32], mean=0, stddev=1/np.sqrt(64)))
b_1 = tf.Variable(tf.random_normal([32], mean=0, stddev=1/np.sqrt(64)))
z_1 = tf.nn.conv2d(X_input, W_1, strides=[1, 1, 1, 1], padding='SAME') + b_1
h_1 = tf.nn.relu(z_1)

p_1 = tf.nn.max_pool(h_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
p_1_flat = tf.reshape(p_1, [-1, 4*4*32])

W_2 = tf.Variable(tf.random_normal([4*4*32, 300], mean=0, stddev=1/np.sqrt(4*4*32)))
b_2 = tf.Variable(tf.random_normal([300], mean=0, stddev=1/np.sqrt(4*4*32)))
z_2 = tf.matmul(p_1_flat, W_2) + b_2
h_2 = tf.nn.relu(z_2)

W_3 = tf.Variable(tf.random_normal([300, 10], mean=0, stddev=1/np.sqrt(300)))
b_3 = tf.Variable(tf.random_normal([10], mean=0, stddev=1/np.sqrt(300)))
z_3 = tf.matmul(z_2, W_3) + b_3
y_ = tf.nn.softmax(z_3)

init = tf.initialize_all_variables()

cost_function = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(0.0001).minimize(cost_function)

correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(2000):
        _, cost = sess.run([optimizer, cost_function], feed_dict={X: X_train, Y: y_train_enc})
        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={X: X_train, Y: y_train_enc})
            print("step %d, training accuracy %g, %g"%(i, cost/X_train.shape[0], train_accuracy))

    test_accuracy = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_enc})
    print("test accuracy %g" % (test_accuracy))

step 0, training accuracy 2.33419, 0.134818
step 100, training accuracy 0.179741, 0.962894
step 200, training accuracy 0.0912966, 0.977118
step 300, training accuracy 0.0587256, 0.990105
step 400, training accuracy 0.0414473, 0.994434
step 500, training accuracy 0.0309149, 0.996289
step 600, training accuracy 0.0239119, 0.996908
step 700, training accuracy 0.0189934, 0.997526
step 800, training accuracy 0.015393, 0.998763
step 900, training accuracy 0.0126936, 0.998763
step 1000, training accuracy 0.0106381, 0.999382
step 1100, training accuracy 0.00905095, 1
step 1200, training accuracy 0.00780718, 1
step 1300, training accuracy 0.00681511, 1
step 1400, training accuracy 0.00601186, 1
step 1500, training accuracy 0.00535226, 1
step 1600, training accuracy 0.00480379, 1
step 1700, training accuracy 0.0043428, 1
step 1800, training accuracy 0.00395103, 1
step 1900, training accuracy 0.00361488, 1
test accuracy 0.994444
